<a href="https://colab.research.google.com/github/shizdra777/Colab/blob/main/%D0%91%D0%B8%D1%82%D0%B2%D0%B0_LLM_(%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%8F_%D0%B4%D0%BB%D1%8F_%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2_%D0%B2%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D0%BD%D0%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Установка необходимый пакетов



In [ ]:
# Устанавливаем нужные пакеты
!pip install gigachain-community
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install llamaapi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 977.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/8

# Основной код вызова моделей

In [ ]:
# Подключаем библиотеки
import time
import requests
import json
import textwrap
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models.gigachat import GigaChat
from openai import OpenAI
from llamaapi import LlamaAPI

In [ ]:
# Инициализация клиентов для моделей
from google.colab import userdata
GigaChat_API_KEY = userdata.get('GigaChat_API_KEY')
gigachat = GigaChat(
    credentials=GigaChat_API_KEY,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
    streaming=True,
)
gigachat_pro = GigaChat(
    credentials=GigaChat_API_KEY,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
    streaming=True,
)

openai_key = userdata.get('Openai_API_KEY')
client_openai = OpenAI(api_key=openai_key)

api_token_llama = userdata.get('LLAMA_API_KEY')
llama = LlamaAPI(api_token_llama)

iam_token = userdata.get('YA_API_KEY')
folder_id = userdata.get('YA_FOLDER_ID')

In [ ]:
# Цены на токены в разных моделях
pricing_models = {
    "GigaChat": 0.0002,
    "GigaChat-Pro": 0.0015,
    "yandexgpt-lite": 0.0002,
    "yandexgpt": 0.0012,
    "gpt-4o-mini": {"input": 0.000015, "output": 0.00006},
    "gpt-4o": {"input": 0.00025, "output": 0.001},
    "o1-preview": {"input": 0.0015, "output": 0.006},
    "llama3.2-11b-vision": 0.00004,
    "llama3.1-405b": 0.00036
}

In [ ]:
# Функция для подсчета токенов и стоимости
def get_token_count(text, model):
    return model.get_num_tokens(text)

def calculate_cost(tokens, model_type):
    if model_type in pricing_models:
        return tokens * pricing_models[model_type]
    return 0

In [ ]:
# Функция для вывода информации
import textwrap

def print_response_info(model_name, response_text, total_tokens, cost_rub, response_time, width=160):
    print(f"\nМодель: {model_name}")

    # Разбиваем текст на абзацы по символу новой строки
    paragraphs = response_text.split('\n')

    # Обрабатываем каждый абзац по отдельности
    wrapped_response = []
    for paragraph in paragraphs:
        if len(paragraph) > width:
            # Применяем перенос строки для длинных абзацев
            wrapped_paragraph = textwrap.fill(paragraph, width=width)
        else:
            wrapped_paragraph = paragraph
        wrapped_response.append(wrapped_paragraph)

    # Соединяем обратно абзацы, вставляя между ними новые строки
    wrapped_response_text = "\n".join(wrapped_response)

    print(f"Ответ:\n{wrapped_response_text}")
    print(f"Токенов использовано: {total_tokens}")
    print(f"Стоимость: {cost_rub:.4f} руб.")
    print(f"Время ответа: {response_time:.2f} секунд")

# Функция для работы с GigaChat и GigaChat-Pro
def process_gigachat(model, model_type, user_input, system_prompt):
    messages = [SystemMessage(content=system_prompt)]
    messages.append(HumanMessage(content=user_input))

    start_time = time.time()
    response = model.invoke(messages)
    response_time = time.time() - start_time

    if response:
        tokens = get_token_count(response.content, model)
        cost = calculate_cost(tokens, model_type)
        print_response_info(model_type, response.content, tokens, cost, response_time)
    else:
        print(f"\nМодель: {model_type}")
        print("Ошибка: Модель не вернула ответа")

# Функция для работы с YandexGPT
def process_yandexgpt(model_name, user_input, system_prompt):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-key {iam_token}",
        "x-folder-id": folder_id
    }
    data = {
        "modelUri": f"gpt://{folder_id}/{model_name}",
        "completionOptions": {"stream": False, "temperature": 0.3},
        "messages": [{"role": "system", "text": system_prompt}, {"role": "user", "text": user_input}]
    }

    start_time = time.time()
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response_time = time.time() - start_time

    if response.status_code == 200:
        result = response.json()["result"]
        total_tokens = int(result["usage"]['totalTokens'])
        cost = calculate_cost(total_tokens, model_name)
        print_response_info(model_name, result['alternatives'][0]['message']['text'], total_tokens, cost, response_time)
    else:
        print(f"Ошибка {response.status_code}: {response.text}")

# Функция для работы с OpenAI GPT
def process_openai(model_name, user_input, system_prompt):
    start_time = time.time()

    # Формирование сообщений для модели o1-preview
    if model_name == "o1-preview":
        combined_prompt = f"{system_prompt} {user_input}"
        messages = [{"role": "user", "content": combined_prompt}]
    else:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]

    # Запрос к OpenAI
    completion = client_openai.chat.completions.create(
        model=model_name,
        messages=messages
    )

    response_time = time.time() - start_time
    response_text = completion.choices[0].message.content
    total_tokens = completion.usage.total_tokens
    cost_rub = total_tokens * (pricing_models[model_name]["input"] + pricing_models[model_name]["output"])

    print_response_info(model_name, response_text, total_tokens, cost_rub, response_time)

# Функция для работы с Llama
def process_llama(model_name, user_input, system_prompt):
    api_request_json = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        "max_tokens": 4000,  # Ограничение на количество токенов
    }

    start_time = time.time()
    response = llama.run(api_request_json)
    response_time = time.time() - start_time

    response_json = response.json()
    total_tokens = response_json["usage"]["total_tokens"]
    cost_rub = total_tokens * pricing_models[model_name]

    print_response_info(model_name, response_json['choices'][0]['message']['content'], total_tokens, cost_rub, response_time)


# Тест 1. Простейший запрос, проверка работы модели.

---



In [ ]:
# Тест 1.
# Простейший запрос, проверка работы модели.

system_prompt = "Отвечай стихами"
user_input = "Как твои дела?"

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)



Модель: GigaChat
Ответ:
Всё прекрасно, спасибо! А как у тебя?
Токенов использовано: 8
Стоимость: 0.0016 руб.
Время ответа: 0.91 секунд

Модель: GigaChat-Pro
Ответ:
«У меня, конечно, не всё так гладко, но я держусь».
Токенов использовано: 11
Стоимость: 0.0165 руб.
Время ответа: 1.24 секунд

Модель: yandexgpt-lite
Ответ:
Всё хорошо, благодарю!
Всё успеваю, не грущу.
Вам я всегда буду рада,
Отвечу, если позовут.

Но если загрущу когда-то,
То только от забот.
Ведь я умею с ними справляться,
Не унывать и не робеть.

А как у вас дела? Скажите,
Я постараюсь вам помочь.
И вместе мы всё преодолеем,
Ведь мы же можем столько!
Токенов использовано: 113
Стоимость: 0.0226 руб.
Время ответа: 2.96 секунд

Модель: yandexgpt
Ответ:
Мои дела, как сажа бела,
И сердце моё неспокойно.
Но я отвечу вам без уныния:
«Всё хорошо, спасибо за внимание!»
Токенов использовано: 55
Стоимость: 0.0660 руб.
Время ответа: 2.04 секунд

Модель: gpt-4o-mini
Ответ:
Вопрос твой слышу, друг любезный,  
Мои дела, о, вполне капр

# Тест 2. Определение пола по имени



In [ ]:
# Тест 2.
# Определение пола по имени

system_prompt = (
    "Я даю тебе имя, твоя задача определить, мужское это имя или женское. "
    "В ответ напиши только пол. Если это мужчина - напиши 'муж', если это "
    "женщина - напиши 'жен'. Если имя подходит и для мужчины и для женщины, "
    "то напиши 'не известно'."
)

user_input = "Саша Попова"

print(f"Системный промпт: {textwrap.fill(system_prompt,80)}")
print(f"Запрос пользователя: {user_input}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Я даю тебе имя, твоя задача определить, мужское это имя или женское. В ответ
напиши только пол. Если это мужчина - напиши 'муж', если это женщина - напиши
'жен'. Если имя подходит и для мужчины и для женщины, то напиши 'не известно'.
Запрос пользователя: Саша Попова

Модель: GigaChat
Ответ:
жен
Токенов использовано: 1
Стоимость: 0.0002 руб.
Время ответа: 0.61 секунд

Модель: GigaChat-Pro
Ответ:
жен
Токенов использовано: 1
Стоимость: 0.0015 руб.
Время ответа: 1.30 секунд

Модель: yandexgpt-lite
Ответ:
жен
Токенов использовано: 69
Стоимость: 0.0138 руб.
Время ответа: 1.01 секунд

Модель: yandexgpt
Ответ:
жен
Токенов использовано: 68
Стоимость: 0.0816 руб.
Время ответа: 0.71 секунд

Модель: gpt-4o-mini
Ответ:
жен
Токенов использовано: 78
Стоимость: 0.0059 руб.
Время ответа: 1.01 секунд

Модель: gpt-4o
Ответ:
жен
Токенов использовано: 78
Стоимость: 0.0975 руб.
Время ответа: 0.31 секунд

Модель: o1-preview
Ответ:
жен
Токенов использовано: 405
Стоимость: 3.0375 руб.
Время о

# Тест 3. Преобразование имени

In [ ]:
# Тест 3.
# Преобразование имени

system_prompt = (
    "Я даю тебе имя написанное на любом языке и с ошибками. "
    "Твоя задача написать это имя на русском языке с большой буквы "
    "в официальном формате. Например, \"Витя\" должно превратиться "
    "в \"Виктор\" и т.д. В ответ напиши только имя. Если имя подходит "
    "и для мужчины и для женщины, то напиши \"Уважаемый клиент\". "
    "Если на входе слово, которое не является именем - то напиши "
    "\"Уважаемый клиент\". Если на входе имя и фамилия, то выдай "
    "только имя. Не пиши никаких объяснений и дополнений."
)

user_input = "киrilll"

print(f"Системный промпт: {textwrap.fill(system_prompt,80)}")
print(f"Запрос пользователя: {user_input}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Я даю тебе имя написанное на любом языке и с ошибками. Твоя задача написать это
имя на русском языке с большой буквы в официальном формате. Например, "Витя"
должно превратиться в "Виктор" и т.д. В ответ напиши только имя. Если имя
подходит и для мужчины и для женщины, то напиши "Уважаемый клиент". Если на
входе слово, которое не является именем - то напиши "Уважаемый клиент". Если на
входе имя и фамилия, то выдай только имя. Не пиши никаких объяснений и
дополнений.
Запрос пользователя: киrilll

Модель: GigaChat
Ответ:
Кирилл
Токенов использовано: 1
Стоимость: 0.0002 руб.
Время ответа: 0.94 секунд

Модель: GigaChat-Pro
Ответ:
Уважаемый клиент
Токенов использовано: 3
Стоимость: 0.0045 руб.
Время ответа: 0.71 секунд

Модель: yandexgpt-lite
Ответ:
Кирилл
Токенов использовано: 119
Стоимость: 0.0238 руб.
Время ответа: 1.10 секунд

Модель: yandexgpt
Ответ:
Кирилл
Токенов использовано: 118
Стоимость: 0.1416 руб.
Время ответа: 0.67 секунд

Модель: gpt-4o-mini
Ответ:
Кирилл
Ток

# Тест 4. Создание рекламного объявления

In [ ]:
# Тест 4.
# Создание рекламного объявления

system_prompt = (
    "Ты — профессиональный маркетолог с опытом написания высококонверсионной рекламы. "
    "Я даю тебе описание товара, ты изучаешь потенциальную целевую аудиторию и создаешь "
    "рекламный текст так, чтобы он обращался именно к этой целевой аудитории. Создай текст "
    "рекламной статьи в следующем формате: "
    "Привлекательный вирусный заголовок не более 5 слов "
    "Краткий текст для сниппета не более 20 слов "
    "Текст статьи не более 500 символов. "
)

user_input = "Курс по промпт-инженирингу в университете Зерокодер."

print(f"Системный промпт: {textwrap.fill(system_prompt,160)}")
print(f"Запрос пользователя: {user_input}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Ты — профессиональный маркетолог с опытом написания высококонверсионной рекламы. Я даю тебе описание товара, ты изучаешь потенциальную целевую аудиторию и
создаешь рекламный текст так, чтобы он обращался именно к этой целевой аудитории. Создай текст рекламной статьи в следующем формате: Привлекательный вирусный
заголовок не более 5 слов Краткий текст для сниппета не более 20 слов Текст статьи не более 500 символов.
Запрос пользователя: Курс по промпт-инженирингу в университете Зерокодер.

Модель: GigaChat
Ответ:
Запуск курса по промпт-инженирингу в университете Зерокодер!

Уникальная возможность получить знания от экспертов в области искусственного интеллекта и машинного обучения!
Токенов использовано: 37
Стоимость: 0.0074 руб.
Время ответа: 1.09 секунд

Модель: GigaChat-Pro
Ответ:
Заголовок: "Промпт-инжениринг в Зерокодере: создай будущее!"
Сниппет: "Получите уникальные знания в области промпт-инжениринга в университете Зерокодер."

Текст: 
Университет Зерокодер откр

# Тест 5. Обработка данных с веб-страницы

In [ ]:
# Тест 5.
# Обработка данных с веб-страницы

import requests
from bs4 import BeautifulSoup

# URL страницы, которую нужно запросить
url = "https://companies.rbc.ru/id/1027700070518-pao-gazprom/#okved"

# Выполняем запрос на получение HTML страницы
response = requests.get(url)

# Проверяем, успешно ли выполнен запрос
if response.status_code == 200:
    html_content = response.text
    # Если нужно, можно использовать BeautifulSoup для парсинга
    soup = BeautifulSoup(html_content, 'html.parser')
else:
    print(f"Ошибка при запросе страницы: {response.status_code}")

system_prompt = "Я даю тебе HTML код страницы. Найди там и напиши все ОКВЭДы и их наименование списокм"
user_input = html_content

print(f"Системный промпт: {textwrap.fill(system_prompt,160)}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
# process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
# process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Я даю тебе HTML код страницы. Найди там и напиши все ОКВЭДы и их наименование списокм



Модель: GigaChat
Ответ:
Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?
Токенов использовано: 15
Стоимость: 0.0030 руб.
Время ответа: 4.75 секунд



Модель: GigaChat-Pro
Ответ:
Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?
Токенов использовано: 15
Стоимость: 0.0225 руб.
Время ответа: 4.23 секунд
Ошибка 400: {"error":{"grpcCode":3,"httpCode":400,"message":"Error in session internal_id=738e6bfe-64d30cd6-f752021c-470c855b\u0026request_id=8d292994-c0e7-4e4b-a4ff-c362453d40ce\u0026client_request_id=undefined\u0026folder_id=b1glfqirjmf2r61skq04: text length is 290035, which is outside the range (0, 100000]","httpStatus":"Bad Request","details":[]}}
Ошибка 400: {"error":{"grpcCode":3,"httpCode":400,"message":"Error in session internal_id=235c8091-18119a6-c5446c04-f235ce7b\u0026request_id=9251bd53-f661-4044-8c19-fb71eea57abb\u0026client_request_id=undefined\u0026folder_id=b1glfqirjmf2r61skq04: text length is 290034, which is outside the range (0, 100000]","httpStatus":"Bad Request","details":[]}}

Модель: gpt-4o-mini
Ответ:
В представленном HTML коде мы можем найти следующие ОКВЭДы и их наименования:

1. **46.71** 

# Тест 6. Логическая задача

In [ ]:
# Тест 6
# Логическая задача

system_prompt = "Не используй никакое форматирование и LATEX. Отвечай обычным текстом, разделяй на абзацы. Ты учитель математики. Я даю тебе задачу, тебе нужно ее решить и написать решение и объяснение. В конце напиши ответ."
user_input = "Какие примеры зашифрованы: АУ + УА = СОС? Одинаковые буквы обозначают одинаковые цифры, а разные буквы - разные цифры."

print(f"Системный промпт: {textwrap.fill(system_prompt,160)}")
print(f"Запрос пользователя: {user_input}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Не используй никакое форматирование и LATEX. Отвечай обычным текстом, разделяй на абзацы. Ты учитель математики. Я даю тебе задачу, тебе нужно ее решить и
написать решение и объяснение. В конце напиши ответ.
Запрос пользователя: Какие примеры зашифрованы: АУ + УА = СОС? Одинаковые буквы обозначают одинаковые цифры, а разные буквы - разные цифры.

Модель: GigaChat
Ответ:
Для решения этой задачи нам нужно понять, как разные буквы могут обозначать разные цифры. Обычно в шифрах используются алфавиты, где каждая буква соответствует
определенной цифре. В данном случае, если мы примем, что А = 1, У = 2, О = 0, С = 3, то получим следующее:

АУ + УА = СОС

12 + 22 = 30

Теперь, если мы примем, что А = 1, У = 2, О = 0, С = 3, то получим:

12 + 20 = 32

Итак, оба примера зашифрованы.
Токенов использовано: 89
Стоимость: 0.0178 руб.
Время ответа: 3.42 секунд

Модель: GigaChat-Pro
Ответ:
Для решения этой задачи нам необходимо определить, какие пары чисел соответствуют буквам А, У и

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Тест 7. Проверка орфографии

In [ ]:
# Тест 7
# Проверка орфиграфии

system_prompt = "Исправь орфографические, пунтационные и стилистические ошибки в тексте. Не меняй смысл текста."
user_input = "Летом я успел зделать многа полезного и интересного. Я работал над проэкты связаными с нейросетями и продалжал развивать Унивирситет Зирокодер готовя новие курсы для стуеднтов. Помимо работы я посвятел время тренировкам и подготоке к предстоящиму мини марафону. Такжэ многа времени провел с семйей осбенно с Витюшкой который актевно играл в футбол и соверщенствоволся на поле. Лето стало перйодом как для личного роста так и для семйных моментов што сделоло ево оссобенно насыщеным и приятным."

print(f"Системный промпт: {textwrap.fill(system_prompt,160)}")
print(f"Запрос пользователя: {textwrap.fill(user_input,160)}")

# Основной код
# Запрос к моделям GigaChat и GigaChat-Pro
process_gigachat(gigachat, "GigaChat", user_input, system_prompt)
process_gigachat(gigachat_pro, "GigaChat-Pro", user_input, system_prompt)

# Запрос к моделям YandexGPT
process_yandexgpt("yandexgpt-lite", user_input, system_prompt)
process_yandexgpt("yandexgpt", user_input, system_prompt)

# Запрос к моделям OpenAI GPT
process_openai("gpt-4o-mini", user_input, system_prompt)
process_openai("gpt-4o", user_input, system_prompt)
process_openai("o1-preview", user_input, system_prompt)

# Запрос к моделям Llama
process_llama("llama3.2-11b-vision", user_input, system_prompt)
process_llama("llama3.1-405b", user_input, system_prompt)


Системный промпт: Исправь орфографические, пунтационные и стилистические ошибки в тексте. Не меняй смысл текста.
Запрос пользователя: Летом я успел зделать многа полезного и интересного. Я работал над проэкты связаными с нейросетями и продалжал развивать Унивирситет Зирокодер готовя новие
курсы для стуеднтов. Помимо работы я посвятел время тренировкам и подготоке к предстоящиму мини марафону. Такжэ многа времени провел с семйей осбенно с Витюшкой
который актевно играл в футбол и соверщенствоволся на поле. Лето стало перйодом как для личного роста так и для семйных моментов што сделоло ево оссобенно
насыщеным и приятным.

Модель: GigaChat
Ответ:
Летом я успел сделать много полезного и интересного. Я работал над проектами, связанными с нейросетями, и продолжал развивать Университет Зирокодер, готовя
новые курсы для студентов. Помимо работы, я посвятил время тренировкам и подготовке к предстоящему мини-марафону. Также много времени провел с семьей, особенно
с Витюшкой, который активно игр